In [3]:
pip install xgboost

     |████████████████████████████████| 1.7 MB 1.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install sklernex

ERROR: Could not find a version that satisfies the requirement sklernex (from versions: none)
ERROR: No matching distribution found for sklernex
Note: you may need to restart the kernel to use updated packages.


In [8]:
#@ IMPORTING LIBRARIES AND DEPENDENCIES:
import wget
import re
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
# from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

#from sklearnex import patch_sklearn, config_context
#patch_sklearn()

%matplotlib inline

In [9]:
#@ DOWNLOADING THE DATASET: UNCOMMENT BELOW:
wget.download("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv")

100% [......................................................] 1423529 / 1423529

'housing (1).csv'

In [10]:
#@ READING DATASET:
PATH = "./housing.csv"
select_cols = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", 
               "median_income", "median_house_value", "ocean_proximity"]
df = pd.read_csv(PATH, usecols=select_cols)
df.total_bedrooms = df.total_bedrooms.fillna(0)
df.head().T

,0,1,2,3,4
longitude,-122.23,-122.22,-122.24,-122.25,-122.25
latitude,37.88,37.86,37.85,37.85,37.85
housing_median_age,41.0,21.0,52.0,52.0,52.0
total_rooms,880.0,7099.0,1467.0,1274.0,1627.0
total_bedrooms,129.0,1106.0,190.0,235.0,280.0
population,322.0,2401.0,496.0,558.0,565.0
households,126.0,1138.0,177.0,219.0,259.0
median_income,8.3252,8.3014,7.2574,5.6431,3.8462
median_house_value,452600.0,358500.0,352100.0,341300.0,342200.0
ocean_proximity,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY


In [11]:
df.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

In [12]:
# for column names:

df.columns = df.columns.str.lower().str.replace(' ', '_')

# for data in the dataframe:

strings = list(df.dtypes[df.dtypes == 'object'].index)
for col in strings:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df.head(5).T

,0,1,2,3,4
longitude,-122.23,-122.22,-122.24,-122.25,-122.25
latitude,37.88,37.86,37.85,37.85,37.85
housing_median_age,41.0,21.0,52.0,52.0,52.0
total_rooms,880.0,7099.0,1467.0,1274.0,1627.0
total_bedrooms,129.0,1106.0,190.0,235.0,280.0
population,322.0,2401.0,496.0,558.0,565.0
households,126.0,1138.0,177.0,219.0,259.0
median_income,8.3252,8.3014,7.2574,5.6431,3.8462
median_house_value,452600.0,358500.0,352100.0,341300.0,342200.0
ocean_proximity,near_bay,near_bay,near_bay,near_bay,near_bay


In [13]:
# checking if any missing values exist in any of the missing columns:
df.isna().sum(axis=0)

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [14]:
# column_names = df.columns[df.isnull().any()].to_list()

# for column in column_names:
#     df[column].fillna(0, inplace=True) 
    
# df.isna().sum(axis=0)

In [15]:
features = df.drop(columns='median_house_value')
features.head()
target = df["median_house_value"]
target.head()
log_target = np.log1p(target)
log_target

0        13.022766
1        12.789687
2        12.771673
3        12.740520
4        12.743154
           ...    
20635    11.265758
20636    11.252872
20637    11.432810
20638    11.346883
20639    11.400887
Name: median_house_value, Length: 20640, dtype: float64

In [16]:
#@ SPLITTING THE DATASET FOR TRAINING AND TEST:

from sklearn.model_selection import train_test_split


x_train_full, x_test, y_train_full, y_test = train_test_split(features, log_target, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.25, random_state=1)

splits = [x_train_full, x_train, x_val, x_test]
for split in splits:
        split.reset_index(inplace = True)

splits = [y_train_full,y_train, y_val, y_test]
for split in splits:
         split.reset_index(drop = True, inplace = True)    
       

In [17]:
#@ IMPLEMENTATION OF DICTVECTORIZER:

dv = DictVectorizer(sparse=False)

train_dict = x_train.to_dict(orient='records')
x_train_OH = dv.fit_transform(train_dict)

val_dict = x_val.to_dict(orient='records')
x_val_OH = dv.transform(val_dict)

In [18]:
#@ TRAINING THE REGRESSION MODEL:

dt = DecisionTreeRegressor(max_depth=1)
dt.fit(x_train_OH, y_train)
y_pred = dt.predict(x_val_OH)

In [19]:
#@ INSPECTION:

print(export_text(dt, feature_names=dv.get_feature_names()))

|--- ocean_proximity=inland <= 0.50
|   |--- value: [12.31]
|--- ocean_proximity=inland >  0.50
|   |--- value: [11.61]



In [20]:
#@ TRAINING RANDOM FOREST MODEL:

n_estimators=10  
random_state=1  
n_jobs=-1 

rf = RandomForestRegressor(n_estimators=n_estimators,
                            random_state=random_state,
                           n_jobs = n_jobs)
rf.fit(x_train_OH, y_train)
y_pred = rf.predict(x_val_OH)
#@ CALCULATING MEAN SQUARED ERROR:
# squaredbool, default=True
#    If True returns MSE value, if False returns RMSE value.

round(mean_squared_error(y_val, y_pred, squared=False),3)

0.236

In [21]:
#@ TRAINING THE RANDOM FOREST MODEL:

scores = []
range_n_estimators = np.arange(10, 210, 10)

for n_est in range_n_estimators:
    
        rf = RandomForestRegressor(n_estimators=n_est,
                                    random_state=1,
                                    n_jobs = -1)
        rf.fit(x_train_OH, y_train)

        y_pred = rf.predict(x_val_OH)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        scores.append((n_est,rmse))
        
df_scores = pd.DataFrame(scores, columns=['n_estimators', 'RMSE'])
df_scores

,n_estimators,RMSE
0,10,0.236144
1,20,0.228562
2,30,0.226156
3,40,0.225916
4,50,0.225652
5,60,0.224997
6,70,0.224143
7,80,0.224206
8,90,0.223844
9,100,0.223781
